In [4]:
import flask
from flask import Flask,request
from flask_cors import CORS
from flask import Flask, render_template, Response, request
import cv2
import time
import mediapipe as mp

def VF_FLASK():
    face_model_path = 'haarcascade_frontalface_alt.xml'
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+face_model_path) 
    eye_model_path = 'haarcascade_eye.xml'
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+eye_model_path) 
    cap2 = cv2.VideoCapture('aaa.mp4')
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    cnt=1
    b = True
    x1=0
    y1=0
    w1=0
    h1=0

    #첫번째 와일문 openCV와 mediapipe 좌표 비율 뽑아내기
    while True:
        with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:
            ret, image = cap2.read()
            image = cv2.resize(image, dsize=(928, 524))
            faces = face_cascade.detectMultiScale(image, #찾고 싶은 이미지
                                         1.2, #스케일 정도, 
                                              #검출이 한번 끝난후 다시 1.3배만큼 커널을 키워서 다시 검출 시작
                                         7) #중첩횟수

            if faces==():
                #영상 안나오면 출력
                print('1')
            else:
                for (x,y,w,h) in faces:
                    x1=x
                    y1=y
                    w1=w
                    h1=h
                    print(x1,y1,w1,h1)
                    results = pose.process(image)
                    if results.pose_landmarks != None:
                        noseX = float(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x)
                        noseY = float(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y)
                    x=x1/noseX
                    y=y1/noseY
            image.flags.writeable = True
            k=cv2.waitKey(1)
            if ret==False or x1!=0: #입력한 key가 ESC인지 판별
                #cv2.destroyAllWindows()
                cap2.release()#카메라 메모리 해제
                break

    #두번째 와일문 얼굴에 사각형 그려주기
    cap1 = cv2.VideoCapture('aaa.mp4')

    face_mask = cv2.imread('ryan_transparent.png')
    with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:
        while True:
            ret, image = cap1.read()
            image = cv2.resize(image, dsize=(928, 524))
            results = pose.process(image)
            if results.pose_landmarks != None:
                    noseX = float(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x)
                    noseY = float(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y)
                    cv2.rectangle(image,(int(noseX*x),int(noseY*y)),(int(noseX*x)+w1,int(noseY*y)+h1),
                                  (0,0,255),3)
                    if h1 > 0 and w1 > 0:
                        x2 = int(int(noseX*x)-w1*0.1 )
                        y2 = int(int(noseY*y) -h1*0.05)
                        w2 = int(1.2* w1)
                        h2 = int(1.2 * h1)
                        frame_roi = image[y2:y2 + h2, x2:x2 + w2]
                        face_mask_small = cv2.resize(face_mask, (w2, h2), interpolation=cv2.INTER_AREA)
                        gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                        ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_BINARY_INV)
                    mask_inv = cv2.bitwise_not(mask)
                    masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                    masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                    image[y2:y2 + h2, x2:x2 + w2] = cv2.add(masked_face, masked_frame)
                    
                    ret1, buffer = cv2.imencode('.jpg', image)
                    vfframe = buffer.tobytes()
                    yield (b'--frame\r\n'
                           b'Content-Type: image/jpeg\r\n\r\n' + vfframe + b'\r\n')
                    
                    #cv2.imshow('Blended Image1',image)
                    k=cv2.waitKey(1)
                    if k ==27 or ret==False: #입력한 key가 ESC인지 판별
                     #   cv2.destroyAllWindows()
                        cap1.release()#카메라 메모리 해제
                        break

In [ ]:
def acc_img(file):
    image = cv2.imread(f'number_data/{file}.png',#이미지 경로
                cv2.IMREAD_COLOR #이미지의 컬러 또는 흑백으로 불러오기
                )
    ret, buffer = cv2.imencode('.jpg', image)
    frame = buffer.tobytes()
    yield (b'--frame\r\n'
       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [ ]:
app = Flask(__name__)
@app.route('/vf')
def send_img():
    
    return Response(VF_FLASK(), mimetype='multipart/x-mixed-replace; boundary=frame')
if __name__ == "__main__":
    app.run("127.0.2.1",port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.2.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Aug/2021 14:57:09] "GET /video_feed HTTP/1.1" 404 -


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
459 111 44 44


C:\Users\smhrd\anaconda3\envs\Deep_GPU2021\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
127.0.0.1 - - [10/Aug/2021 14:57:26] "GET /vf HTTP/1.1" 200 -
